In [28]:
import glob
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
import h5py

I=0
def _parse_function(example_proto):
    print('1')
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    print("i was here")
    print('2')
    return parsed_features["X"], parsed_features["Y"]

def rbm_layer(n_visible, n_hidden, num_epochs, num_cases, lr, ws, bs, layer_n, len_data, directories):
    Data_path = directories[0]
    tfrecord_folder_parent = directories[1]
    tfrecord_folder = directories[2]
    
    tfrecord_path_x = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
    sorted_names_x = natsorted(os.listdir(tfrecord_path_x))
    trainfilenames_x = []
    for i in sorted_names_x:
        trainfilenames_x.append(os.path.normpath(os.path.join(tfrecord_path_x,i)))
#     filenames_x = tf.placeholder(tf.string, shape=[None])
#     dataset_x = tf.data.TFRecordDataset(filenames_x)
    dataset_x = tf.data.TFRecordDataset(trainfilenames_x)
    dataset_x = dataset_x.map(_parse_function)  # Parse the record into tensors.
    dataset_x = dataset_x.repeat()  # Repeat the input indefinitely.
    dataset_x = dataset_x.batch(num_cases)
#     iterator_x = dataset_x.make_initializable_iterator()
    
    weightcost  = 0.0002
#     initialmomentum  = 0.5
#     finalmomentum    = 0.9
    momentum  = 0.5
    numcases = 32
    W_adder  = tf.zeros((n_visible,n_hidden),dtype=tf.dtypes.float32)
    bh_adder = tf.zeros((1,n_hidden),dtype=tf.dtypes.float32)
    bv_adder = tf.zeros((1,n_visible),dtype=tf.dtypes.float32)
#     x  = tf.placeholder(tf.float32, [None, n_visible], name="x") #The placeholder variable that holds our data
#     h = tf.placeholder(tf.float32, [None, n_hidden], name="h")
#     h = tf.zeros((numcases, n_hidden),dtype=tf.dtypes.float32)
#     h_empty  = np.zeros_like(h)
#     h_empty = np.zeros((, n_hidden),dtype=np.float32)
#     m  = tf.Variable(0.5, dtype=np.float32, name='m')
    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W") #The weight matrix that stores the edge weights
    bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
    bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer

    def sample(probs):
        #Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
        return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

#     hk = sample(tf.sigmoid(tf.matmul(x, W) + bh)) #Propagate the visible values to sample the hidden values
#     #128*512

#     #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
#     posprods  = tf.matmul(tf.transpose(x),hk) #771*512
#     poshidacts = tf.reduce_sum(hk) #(512)
#     posvisacts = tf.reduce_sum(x,axis=0)#771
    
#     xk   = tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)#128*711
#     neghidprobs   = tf.sigmoid(tf.matmul(xk, W) + bh) #128*512
#     negprods  = tf.matmul(tf.transpose(xk),neghidprobs); #771*512##################################
#     neghidacts = tf.reduce_sum(neghidprobs, axis=0)
#     negvisacts = tf.reduce_sum(xk, axis=0)
#     print(negvisacts)#771
# #     print(type(posprod-negprod))
# #     print(type(lr))
#     m=0.5
#     print('here')
#     W_adder = (m * W_adder)+ (lr*(posprods-negprods)/numcases)-(weightcost*W)
#     bv_adder = (m * bv_adder)+ ((lr/numcases)*(posvisacts-negvisacts))
#     bh_adder = (m * bh_adder)+ ((lr/numcases)*(poshidacts-neghidacts))
#     #When we do sess.run(updt), TensorFlow will run all 3 update steps
#     updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]

    ### Run the graph!
    # Now it's time to start a session and run the graph! 

#     with tf.Session() as sess:
        #First, we train the model
        #initialize the variables of the model
#         sess.run(tf.global_variables_initializer())
#         print(m)
#         sess.run(iterator_x.initializer, feed_dict={filenames_x: trainfilenames_x})
#         print(iterator_x.get_next()[0])
        #Run through all of the training data num_epochs times
#         for epoch in tqdm(range(num_epochs)):
            #Train the RBM on batch_size examples at a time
#             for X_batch in da(batch_size, layer_n, ws, bs, len_data, name, data_name):
#             Data = sess.run(iterator_x.get_next()[0])
#             print('blah')
#             print(Data.shape)
#             for Data in dataset_x:
#                 for j in range(layer_n):
#                     Data = np.matmul(Data,ws[j])+bs[j]
    #             if epoch>5:
    #                 momentum=finalmomentum;
    #             else:
    #                 momentum=initialmomentum;
#                 feed_dict = {x: Data}
#                 var1 = sess.run(updt, feed_dict)
    count=0
    for epoch in tqdm(range(num_epochs)):
        count+=1
        for x in dataset_x:
            hk = sample(tf.sigmoid(tf.matmul(x[0], W) + bh)) #Propagate the visible values to sample the hidden values
            #128*512
            #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
            posprods  = tf.matmul(tf.transpose(x[0]),hk) #771*512
            poshidacts = tf.reduce_sum(hk) #(512)
            posvisacts = tf.reduce_sum(x[0],axis=0)#771

            xk   = tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)#128*711
            neghidprobs   = tf.sigmoid(tf.matmul(xk, W) + bh) #128*512
            negprods  = tf.matmul(tf.transpose(xk),neghidprobs); #771*512##################################
            neghidacts = tf.reduce_sum(neghidprobs, axis=0)
            negvisacts = tf.reduce_sum(xk, axis=0)
#             print(negvisacts)#771
        #     print(type(posprod-negprod))
        #     print(type(lr))
            m=0.5
            W_adder = (m * W_adder)+ (lr*(posprods-negprods)/numcases)-(weightcost*W)
            bv_adder = (m * bv_adder)+ ((lr/numcases)*(posvisacts-negvisacts))
            bh_adder = (m * bh_adder)+ ((lr/numcases)*(poshidacts-neghidacts))
            #When we do sess.run(updt), TensorFlow will run all 3 update steps
            updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)] 
        print(count)
    return updt




In [ ]:
# import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
tf.enable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from pystoi.stoi import stoi
import h5py
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = os.getcwd()
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'tfrecord_files_10h_norm'
tfrecord_val_folder = 'validation_10h_norm'
ckpt_folder = '3'
dirs = [Data_path, tfrecord_folder_parent, tfrecord_folder]
 
# len_data = (684108, 257)
len_data = (100000, 257)
val_len = (97278,257)
w=3
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=200, win_length=500, window='hann')
    return recon
#######################
I=0
global batch_size
batch_size = 32
# epochs_num=50
global datalen
datalen=len_data[0]

h = [512,512]
seed = 7
rate1 = 0.1
rate2 = 0.2
from tensorflow.keras.layers import Activation
# from keras.layers import Activation
np.random.seed(seed)
model = Sequential()
act1 = layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate1, noise_shape=None, seed=None))
# ,kernel_regularizer=regularizers.l2(0.001)
model.add(Dense(h[0], input_dim = w*len_data[1]))
model.add(BatchNormalization())
model.add(act1)
# model.add(Activation('sigmoid'))
act2=layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate2, noise_shape=None, seed=None))
model.add(Dense(h[1]))
model.add(act2)
# act3=layers.LeakyReLU(alpha=0.1)
# # model.add(layers.Dropout(rate, noise_shape=None, seed=None))
# model.add(Dense(h[2]))
# model.add(act3)
act=layers.LeakyReLU(alpha=0.01)
model.add(Dense(len_data[1]))
#############################################
import os
from natsort import natsorted

# def _parse_function(example_proto):
#     features = {"X": tf.FixedLenFeature((3*257), tf.float32),
#               "Y": tf.FixedLenFeature((257), tf.float32)}
#     parsed_features = tf.parse_single_example(example_proto, features)
#     return parsed_features["X"], parsed_features["Y"]

# tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
# sorted_names = natsorted(os.listdir(tfrecord_path))
# trainfilenames = []
# for i in sorted_names:
#     trainfilenames.append(os.path.normpath(os.path.join(tfrecord_path,i)))
# filenames = tf.placeholder(tf.string, shape=[None])
# dataset = tf.data.TFRecordDataset(filenames)
# dataset = dataset.map(_parse_function)  # Parse the record into tensors.
# dataset = dataset.repeat()  # Repeat the input indefinitely.
# dataset = dataset.batch(batch_size)
# iterator = dataset.make_initializable_iterator()

# # orig_path = os.getcwd()
# tfrecord_path_x = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
# sorted_names_x = natsorted(os.listdir(tfrecord_path_x))
# trainfilenames_x = []
# for i in sorted_names_x:
#     trainfilenames_x.append(os.path.normpath(os.path.join(tfrecord_path,i)))
# filenames_x = tf.placeholder(tf.string, shape=[None])
# dataset_x = tf.data.TFRecordDataset(filenames_x)
# dataset_x = dataset_x.map(_parse_function)  # Parse the record into tensors.
# dataset_x = dataset_x.repeat()  # Repeat the input indefinitely.
# dataset_x = dataset_x.batch(batch_size)
# iterator_x = dataset_x.make_initializable_iterator()

########################
visible = w*len_data[1]
hidden = h[0]
visible1 = h[0]
hidden1 = h[1]
visible2 = h[1]
hidden2 = len_data[1]

layer1 = rbm_layer(visible, hidden, 50, batch_size, 0.0005, [np.eye(visible)], [np.zeros((1,visible))], 1, len_data[0],dirs)
# layer2 = rbm_layer(visible1, hidden1, 50, batch_size, 0.01, [np.eye(visible),layer1[0]], [np.zeros((1,visible)),layer1[2]], 2, len_data[0], dirs)
# layer3 = rbm_layer(visible2, hidden2, 50, batch_size, 0.01, [np.eye(visible),layer1[0],layer2[0]], [np.zeros((1,visible)),layer1[2],layer2[2]], 3, len_data[0], dirs)

###############################

# tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
# sorted_names_val = natsorted(os.listdir(tfrecord_path_val))
# trainfilenames_val = []
# for i in sorted_names_val:
#     trainfilenames_val.append(os.path.normpath(os.path.join(tfrecord_path_val,i)))
# filenames_val = tf.placeholder(tf.string, shape=[None])
# dataset_val = tf.data.TFRecordDataset(filenames_val)
# dataset_val = dataset_val.map(_parse_function)  # Parse the record into tensors.
# dataset_val = dataset_val.repeat()  # Repeat the input indefinitely.
# dataset_val = dataset_val.batch(128)
# iterator_val = dataset_val.make_initializable_iterator()

# epochs_num = 50
# steps = len_data[0] // batch_size
# val_steps = val_len[0] // batch_size
# # You can feed the initializer with the appropriate filenames for the current
# # phase of execution, e.g. training vs. validation.
# # next_elem = iterator_val.get_next()
# # Initialize `iterator` with training data.

# if not os.path.exists(os.path.join(Data_path,"checkpoints",ckpt_folder)):
#     os.makedirs(os.path.join(Data_path,"checkpoints",ckpt_folder))

# print(datetime.datetime.now())
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(iterator.initializer, feed_dict={filenames: trainfilenames})
#     sess.run(iterator_val.initializer, feed_dict={filenames_val: trainfilenames_val})
#     print("initialized")
#     checkpoint_path = os.path.normpath(os.path.join(Data_path,"checkpoints",ckpt_folder,"weights.{epoch:02d}.hdf5"))
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     cp_callback = tf.keras.callbacks.ModelCheckpoint(
#         checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
#         # Save weights, every 5-epochs.
# #         period=1)
#     early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
#     opt = tf.keras.optimizers.Adamax()
# #     opt = tf.train.AdamOptimizer()
# #     opt = tf.keras.optimizers.SGD()
#     model.compile(loss='mean_squared_error', optimizer=opt)
#     history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, callbacks = [cp_callback,early_stop], verbose=1,validation_data=iterator_val,validation_steps=val_steps)
# #     model.save(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
# #     tf.keras.models.save_model(model, os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
# #     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     model_json = model.to_json()
#     with open(os.path.normpath(os.path.join(Data_path, 'models', "model_3.json")), "w") as json_file:
#         json_file.write(model_json)
#     # # serialize weights to HDF5
#     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3.h5")))
#     print("Saved model to disk")
    
# print(datetime.datetime.now())
# %matplotlib inline
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train'], loc='upper left')
# plt.show()
# plt.savefig(os.path.normpath(os.path.join(Data_path,'images',ckpt_folder+'.png')))
# # model_json = model.to_json()
# # with open("model_10h_dataset.json", "w") as json_file:
# #     json_file.write(model_json)
# # model.save_weights("model_10h_dataset.h5")
# # print("Saved model to disk")


imported
1
i was here
2


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

In [254]:
layer1[2]

array([[7.2521896, 7.2731686, 7.255978 , 7.2539644, 7.252147 , 7.2675285,
        7.252169 , 7.2692385, 7.254997 , 7.254992 , 7.261444 , 7.252759 ,
        7.2521086, 7.2590594, 7.2663407, 7.252896 , 7.2520504, 7.2732315,
        7.2643743, 7.255691 , 7.26014  , 7.263299 , 7.252203 , 7.2615795,
        7.253949 , 7.260395 , 7.2520556, 7.2524915, 7.267142 , 7.2523346,
        7.2649045, 7.25217  , 7.2553096, 7.2619376, 7.266017 , 7.2528844,
        7.2769527, 7.2522364, 7.253238 , 7.269687 , 7.2550793, 7.25774  ,
        7.2523484, 7.253027 , 7.258409 , 7.257243 , 7.2587447, 7.255395 ,
        7.270318 , 7.2594857, 7.2520995, 7.268672 , 7.2676725, 7.2521133,
        7.2692537, 7.254949 , 7.258311 , 7.25226  , 7.2743955, 7.2525625,
        7.265415 , 7.2554445, 7.2715683, 7.2532167, 7.255907 , 7.2529187,
        7.275555 , 7.255739 , 7.2581916, 7.255965 , 7.2595234, 7.252056 ,
        7.252051 , 7.253619 , 7.26754  , 7.269498 , 7.2536983, 7.260301 ,
        7.2581143, 7.2526345, 7.256154

In [193]:
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'ftr_refrmd_10h_norm'))
h5f = h5py.File(mixed_folder+'.hdf5','r')
ftr = h5f['ftr_refrmd_10h_norm'][0:126]
h5f.close()

In [271]:
hidden1 = np.matmul(ftr,layer1[0])+layer1[2]
# hidden2 = np.matmul(hidden1,layer2[0])+layer2[2]
# hidden3 = np.matmul(hidden2,layer3[0])+layer3[2]
hidden1_b = np.matmul(hidden1,layer1[0].T)+layer1[1]
# hidden2_b = np.matmul(hidden3_b,layer2[0].T)+layer2[1]
# hidden1_b = np.matmul(hidden2_b,layer1[0].T)+layer1[1]

In [272]:
layer1[0]

array([[-0.9299576 , -0.7363741 , -0.01299872, ..., -0.17403848,
        -0.15316603,  1.1178471 ],
       [ 0.387127  ,  0.17811282, -1.0233028 , ...,  0.76897204,
         0.31726673, -1.6071396 ],
       [ 0.9451304 ,  0.1530272 , -1.1989824 , ...,  0.76721895,
         1.0839846 , -0.23943205],
       ...,
       [ 1.1624964 ,  0.6948519 ,  0.46188653, ...,  1.315947  ,
        -0.6755554 ,  1.4293516 ],
       [ 1.7025452 ,  0.59827197,  1.2019066 , ..., -1.1621488 ,
        -0.25516018, -0.43353298],
       [-0.14567445, -1.1976821 ,  0.3586812 , ...,  0.46719837,
        -2.026935  ,  0.66572046]], dtype=float32)

In [273]:
ftr

array([[ 1.3411713 ,  1.3411713 ,  1.3411713 , ..., -0.67082363,
        -0.6855072 , -1.206295  ],
       [ 5.9091597 ,  5.521963  ,  4.2596936 , ..., -0.26952586,
        -0.01232617,  0.28907207],
       [ 5.593701  ,  5.6730733 ,  4.610169  , ..., -0.4117906 ,
        -1.0919688 , -1.0797864 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.70710677, -0.70710677, -0.70710677, ...,  1.4142135 ,
         1.4142135 ,  1.4142135 ]], dtype=float32)

In [274]:
hidden1_b

array([[  791.22516 ,  1403.8185  ,   344.96936 , ...,   165.61496 ,
          312.95496 ,   -46.464916],
       [ 2987.522   ,  2818.261   ,  1701.1521  , ..., -1097.625   ,
         -941.82965 ,   696.58417 ],
       [ 3090.814   ,  2840.0752  ,  2953.833   , ..., -1645.8604  ,
         -518.17456 ,  -926.0135  ],
       ...,
       [   34.79614 ,   121.58834 ,    73.61811 , ...,   115.97396 ,
          -94.66215 ,   115.987236],
       [   34.79614 ,   121.58834 ,    73.61811 , ...,   115.97396 ,
          -94.66215 ,   115.987236],
       [ -827.8012  , -1122.1414  ,   -92.172356, ...,   437.81903 ,
          628.75653 ,  1888.1415  ]], dtype=float32)

In [27]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

In [97]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

(771,)

In [24]:
file.close()

In [92]:
dataset

<PrefetchDataset shapes: ((?, 257), (?,)), types: (tf.float32, tf.float32)>